## LLM

### DeepSeek Inference

In [ ]:
!pip install flash_attn vllm cleantext better_profanity

In [ ]:
# Основные библиотеки
import os
import io
import json
import ast # парсинг и анализ синтаксических деревьев Python-выражений (например, безопасный eval)
import re
import time
import tempfile
import subprocess   # выполнение внешних процессов из Python
import requests     # HTTP-запросы
import random

import pandas as pd
import torch # PyTorch
from transformers import BertModel, BertTokenizer
from transformers import AutoTokenizer

# vLLM — библиотека для ускоренного инференса LLM (Large Language Models)
from vllm import LLM, SamplingParams

from typing import Optional
from collections import Counter
from multiprocessing import Process, Queue # параллельные процессы и очереди между ними
from tqdm import tqdm

# библиотеки для очистки и фильтрации текста
from cleantext import clean           # очистка текста от мусора, символов, HTML и т.п.
from better_profanity import profanity  # детекция и цензурирование нецензурной лексики

In [ ]:
solutions = pd.read_excel('solutions.xlsx')
tasks = pd.read_excel('tasks.xlsx')
tests = pd.read_excel('tests.xlsx')
tests_with_errors = pd.read_csv('test_solutions_with_flake8.csv')

In [ ]:
solutions = tests_with_errors.merge(tasks[['id', 'description', 'author_solution']], left_on='task_id', right_on='id', how='left', suffixes=('', '_task'))

In [ ]:
solutions

,Unnamed: 0,id,task_id,student_solution,author_comment,author_comment_embedding,message,line_number,line,id_task,description,author_solution
0,0,0,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,NaN,NaN,NaN,0,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
1,1,1,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,SyntaxError: unterminated f-string literal (de...,6.0,NaN,0,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
2,2,2,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,SyntaxError: unterminated string literal (dete...,6.0,NaN,0,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
3,3,3,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,NaN,NaN,NaN,0,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
4,4,4,0,discount = float(input())\nmoney = int(input(...,Ваше предсказание,NaN,NaN,NaN,NaN,0,"Реализуйте программу, которая напечатает стоим...",discount = float(input())\nmoney = int(input(...
...,...,...,...,...,...,...,...,...,...,...,...,...
320,320,725,46,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,NaN,NaN,NaN,46,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
321,321,726,46,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,NaN,NaN,NaN,46,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
322,322,727,46,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,NaN,NaN,NaN,46,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...
323,323,728,46,def success(data):\n \n wins = 0\n mo...,Ваше предсказание,NaN,NaN,NaN,NaN,46,"Напишите функцию, определяющую успешна студия ...",def success(data):\n \n wins = 0\n mo...


In [ ]:
max_model_len, tp_size = 8192, 1 # 8192 токена контекста; tp_size=1 → без tensor-parallel на несколько GPU
model_name = "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

llm = LLM(
    model=model_name,                   # та же строка, что и выше
    tensor_parallel_size=tp_size,       # параллелим веса по tp_size устройствам
    max_model_len=max_model_len,        # ограничение длины входа + генерации
    trust_remote_code=True,             # разрешаем выполнять кастомный __init__.py из репозитория модели
    enforce_eager=True                  # отключаем torch.compile / lazy-mode → проще отладка, меньше RAM-пик
)

sampling_params = SamplingParams(
    temperature=0.0,                    # 0.0 → детерминированный greedy (всегда берём токен с макс. prob)
    max_tokens=256,                     # ограничиваем длину выдачи, чтобы не выйти за пределы
    stop_token_ids=[tokenizer.eos_token_id]  # при встрече <eos> модель останавливается
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 10-19 17:46:29 config.py:107] Replacing legacy 'type' key with 'rope_type'
WARNING 10-19 17:46:38 config.py:395] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 10-19 17:46:38 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outli

model-00003-of-000004.safetensors:   0%|          | 0.00/8.59G [00:00<?, ?B/s]

model-00001-of-000004.safetensors:   0%|          | 0.00/8.59G [00:00<?, ?B/s]

model-00004-of-000004.safetensors:   0%|          | 0.00/5.64G [00:00<?, ?B/s]

model-00002-of-000004.safetensors:   0%|          | 0.00/8.59G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-19 17:48:25 model_runner.py:1067] Loading model weights took 29.3010 GB
WARNING 10-19 17:48:26 fused_moe.py:323] Using default MoE config. Performance might be sub-optimal! Config file not found at /usr/local/lib/python3.10/dist-packages/vllm/model_executor/layers/fused_moe/configs/E=64,N=1408,device_name=NVIDIA_A100-SXM4-40GB.json
INFO 10-19 17:48:29 gpu_executor.py:122] # GPU blocks: 674, # CPU blocks: 606
INFO 10-19 17:48:29 gpu_executor.py:126] Maximum concurrency for 8192 tokens per request: 1.32x


Вспомним, что под капотом это все еще просто генеративная модель, которая предсказывает вероятности следующих токенов. Что делать с этими вероятностями дальше - можно определить с помощью стратегии генерации.

| Стратегия | Описание | Плюсы и минусы |
| --- | --- | --- |
| Greedy Search | Выбирает слово с наивысшей вероятностью как следующее слово в последовательности. | Плюсы: Простота и скорость.<br> Минусы: Может привести к повторяющемуся и несвязному тексту. |
| Семплинг с температурой | Добавляет случайность в выбор слова. Большая температура приводит к большей случайности. | Плюсы: Позволяет исследовать и получать разнообразный результат.<br> Минусы: Высокие температуры могут привести к бессмысленным результатам. |
| Семплинг по ядру (Top-p семплинг) | Выбирает следующее слово из усеченного словаря, "ядра" слов, которые имеют суммарную вероятность, превышающую предустановленный порог (p). | Плюсы: Обеспечивает баланс между разнообразием и качеством.<br> Минусы: Настройка оптимального 'p' может быть затруднительна. |
| Beam Search | Исследует множество гипотез (последовательностей слов) на каждом шаге и сохраняет 'k' наиболее вероятных, где 'k' - ширина луча. | Плюсы: Дает более надежные результаты, чем жадный поиск.<br> Минусы: Может страдать от нехватки разнообразия и приводить к общим ответам. |
| Top-k семплинг | Случайным образом выбирает следующее слово из 'k' слов с самыми высокими вероятностями. | Плюсы: Вводит случайность, увеличивая разнообразие результатов.<br> Минусы: Случайный выбор иногда может привести к менее связному тексту. |
| Нормализация длины | Предотвращает предпочтение модели более коротких последовательностей за счет деления логарифмированных вероятностей на длину последовательности, возведенную в некоторую степень. | Плюсы: Делает более длинные и потенциально более информативные последовательности более вероятными.<br> Минусы: Настройка фактора нормализации может быть сложной. |
| Стохастический Beam Search | Вводит случайность в процесс выбора 'k' гипотез в поиске пучком. | Плюсы: Увеличивает разнообразие в сгенерированном тексте.<br> Минусы: Баланс между разнообразием и качеством может быть сложно управлять. |
| Декодирование с минимальным риском Байеса (MBR) | Выбирает гипотезу (из многих), которая минимизирует ожидаемую потерю для функции потерь. | Плюсы: Оптимизирует результат в соответствии с определенной функцией потерь.<br> Минусы: Вычислительно более сложно и требует хорошо подобранную функциию потерь. |

Референсы:
- [Статья о стратегиях генерации на Huggingface](https://huggingface.co/docs/transformers/generation_strategies)

In [ ]:
# Функция для генерации комментария
def generate_comment(task_description, student_solution, synt_errors):

    """
    Использует DeepSeek-Coder-V2-Lite-Instruct для краткого фидбэка к решению студента.
    Возвращает одну фразу — указание на синтаксическую/логическую ошибку.

    Args:
        task_description : формулировка задачи
        student_solution : код ученика (может содержать ошибки)
        synt_errors      : сообщение Python-интерпретатора, если код даже не запускается
    """

    # system-prompt: задаём стиль и правила ответа
    system_message = '''
    Ты - профессиональный программист и учитель.
    Я пришлю тебе условие задачи, верное решение и мое решение с ошибками.
    Дай крайне короткий ответ в одном предложении об ошибке в решении.
    Ошибки могут быть синтаксические и логические.
    Объясни ошибку через смысл задания. Ни в коем случае не ссылайся на реализацию в верном решении.
    Ни в коем случае не присылай исправленный код решения.
    Используй следующие фразы: "ваш код", "выполняет условия", "попробуйте изменить", "условия задания", "некорректно выполняет", "скорректировать ошибку" или задавай наводящие вопросы, если нужно обратить внимание на ошибку в коде.
    '''

    # user-prompt: содержимое конкретного задания
    user_message = f'''
    Условие задачи:
    {task_description}

    Мое решение с ошибкой:
    {student_solution}

    Синтаксические ошибки в решении:
    {synt_errors}
    '''
    # список сообщений в формате ChatML
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

    # превратим в токены с помощью chat-template токенизатора
    prompt_token_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True) # добавит <|assistant|> ... тег

    # генерируем с детерминированным greedy-декодированием
    outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)

    # берём текст первого (и единственного) ответа
    result = outputs[0].outputs[0].text
    return result

In [ ]:
# Пример задачи
task_description = "Реализуйте программу, которая напечатает стоимость товара после применения скидки."
student_solution = "discount = float(input())\nmoney = int(input())\nprint(money * discount"
synt_errors = "SyntaxError: unterminated string literal"

author_comment = generate_comment(task_description, student_solution, synt_errors)

print(f"Сгенерированный комментарий: {author_comment}")

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.99s/it, est. speed input: 93.15 toks/s, output: 16.78 toks/s]

Сгенерированный комментарий:  В вашем коде отсутствует закрывающая кавычка для строки, которая выводится на экран. Также неправильно используется тип данных для ввода суммы денег.


In [ ]:
file_name = 'obscene_lexicon.txt'
with open(file_name, 'r', encoding='utf-8') as file:
    bad_words = file.read().split('\n')

def clean_answer(text):
    """
    Убираем куски кода и цензурируем непристойную лексику, чтобы комментарий
    гарантированно соответствовал политике модерации.
    """

    # 3.1 удалить фигурный код (fenced blocks) и строчные конструкции Python
    def remove_code_from_text(text):
        code_pattern = re.compile(r'\b(def|class|if|else|elif|for|while|try|except|finally|with|import|from|print|return|break|continue|pass|raise|assert|yield|async|await|lambda|global|nonlocal|del|exec|eval)\b.*', re.DOTALL)
        fenced_code_pattern = re.compile(r'```.*?```', re.DOTALL)
        cleaned_text = code_pattern.sub('', text)
        cleaned_text = fenced_code_pattern.sub('', cleaned_text)
        return cleaned_text

    # 3.2 цензура обсценной лексики
    def remove_obscene_lexicon(text):
        cleaned_text = clean(text, extra_spaces=True, stemming=False, stopwords=True, lowercase=False, stp_lang='english' )
        cleaned_text = profanity.censor(cleaned_text)
        for word in bad_words:
            bad_words_re = re.compile(r'\b(' + word + r')\b', re.IGNORECASE)
            cleaned_text = bad_words_re.sub(lambda match: '***', cleaned_text)
        return cleaned_text

    text = remove_code_from_text(text)
    text = remove_obscene_lexicon(text)
    return text

In [ ]:
author_comments = []

for idx, row in solutions.iterrows():  # solutions — DataFrame с решениями студентов
    task_description = row['description']
    student_solution = row['student_solution']
    synt_errors = row['message'] if not pd.isna(row['message']) else "нет синтаксических ошибок"

    author_comment = generate_comment(task_description, student_solution, synt_errors)
    author_comment = clean_answer(author_comment)
    author_comments.append({"id": row['id'], "author_comment": author_comment})

submit_df = pd.DataFrame(author_comments)

submit_df[['id', 'author_comment']].head()

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.95s/it, est. speed input: 124.82 toks/s, output: 16.97 toks/s]
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.99s/it, est. speed input: 185.77 toks/s, output: 16.04 toks/s]


,id,author_comment
0,0,"Вы некорректно выполняете условия задания, исп..."
1,1,В вашем коде не закрытый кавычки в f-строке. Н...
2,2,В вашем коде не закрытый кавычкими строковый л...
3,3,"Выполняет условия задания, но некорректно выпо..."
4,4,Вы некорректно выполняет условия задания. Вы и...


In [ ]:
# Загрузка модели для эмбеддингов

# вариант RuBERT, обученный по методике Sentence-BERT (SBERT).
model_name = "DeepPavlov/rubert-base-cased-sentence"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def get_sentence_embedding(sentence: str) -> torch.Tensor:
    """CLS-пуллинг: берём эмбеддинг первого токена."""
    inputs = tokenizer(
        text=sentence,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return embedding # shape: (768,)

def embedding2string(embedding: torch.Tensor) -> str:
    """Сериализация в строку (для csv-сабмита)."""
    return " ".join([str(i) for i in embedding.tolist()])

embeddings = []

for comment in tqdm(submit_df['author_comment'], desc="Генерация эмбеддингов"):
    embedding = get_sentence_embedding(comment)
    embedding_str = embedding2string(embedding)
    embeddings.append(embedding_str)

submit_df['author_comment_embedding'] = embeddings

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

Генерация эмбеддингов: 100%|██████████| 325/325 [00:19<00:00, 16.57it/s]


In [ ]:
submit_df

,id,author_comment,author_comment_embedding
0,0,"Вы некорректно выполняете условия задания, исп...",-0.4947304129600525 -1.5721532106399536 -0.184...
1,1,В вашем коде не закрытый кавычки в f-строке. Н...,0.13552772998809814 -1.1494126319885254 0.2206...
2,2,В вашем коде не закрытый кавычкими строковый л...,-0.4213145077228546 -1.3999632596969604 -0.334...
3,3,"Выполняет условия задания, но некорректно выпо...",-0.47263726592063904 -1.4783111810684204 0.006...
4,4,Вы некорректно выполняет условия задания. Вы и...,-0.3666882812976837 -1.8071675300598145 -0.108...
...,...,...,...
320,725,"Выполняет условия задания, но некорректно выпо...",-0.5775066018104553 -1.319191336631775 -0.1548...
321,726,"Ошибка в вашем коде заключается в том, что фун...",-0.25639352202415466 -1.2132973670959473 0.560...
322,727,"Выполняет условия задания, но некорректно выпо...",-0.582847535610199 -1.5500305891036987 -0.1360...
323,728,"Выполняет условия задания, но некорректно выпо...",-0.47922369837760925 -1.4892631769180298 -0.27...


### YandexGPT

In [ ]:
oauth_token = ### вот тут был токен

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
headers = {'Content-Type': 'application/json'}
data = {
    "yandexPassportOauthToken": oauth_token
}

# Обмениваем OAuth → IAM (нужен для подписания запросов к API LLM)
response = requests.post(url, json=data, headers=headers)
iam_token = response.json().get('iamToken')

In [ ]:
# Установка переменных окружения — удобно подтягивать в любом месте кода
os.environ['YANDEX_GPT_IAM_TOKEN'] = iam_token
os.environ['YANDEX_GPT_FOLDER_ID'] = ### id каталога на облаке

In [ ]:
class YandexGPT:
    """
    Мини-SDK для работы с Yandex GPT (lite / pro).
    Позволяет вести диалог: хранит историю сообщений и автоматически добавляет system-prompt.
    """

    model_urls = {
        "lite": "gpt://{}/yandexgpt-lite/latest", # 8К контекста, дешевле
        "pro": "gpt://{}/yandexgpt/latest", # 32К контекста, дороже
    }

    def __init__(
        self,
        token: str,
        folder_id: str,
        model_name: str = "pro",
        system_prompt: Optional[str] = None,
        temperature: float = 0.6,
        max_tokens: int = 2000,
    ) -> None:
        # базовый system-prompt задаёт роль модели
        self.system_prompt = system_prompt
        self.messages = [] # храним историю диалога

        # URL REST-единой точки
        self.api_url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"

        # обязательные HTTP-заголовки: IAM-токен + folder-id
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {token}",
            "x-folder-id": folder_id,
        }
        # задаём модель
        self.model_url = YandexGPT.model_urls[model_name].format(folder_id)

        # опции генерации: temperature, длина, stream/off
        self.completion_options = {
            "stream": False, # хотим получить ответ целиком
            "temperature": temperature,
            "maxTokens": str(max_tokens), # API ждёт строку
        }

    def ask(self, user_message: str, clear_history: bool = True) -> Optional[str]:
        """
        :param user_message: реплика пользователя
        :param clear_history: True → начинаем «новый диалог», иначе продолжаем текущий
        :return: текст ответа ассистента или None при ошибке
        """

        if clear_history:
            self.messages = []
            if self.system_prompt:
                self.messages.append({"role": "system", "text": self.system_prompt})

        # добавляем реплику пользователя в историю
        self.messages.append({"role": "user", "text": user_message})

        json_request = {
            "modelUri": self.model_url,
            "completionOptions": self.completion_options,
            "messages": self.messages,
        }

        response = requests.post(self.api_url, headers=self.headers, json=json_request)
        if response.status_code != 200:
            print("Ошибка:", response.status_code, response.text)
            return None

        response_data = response.json()
        assistant_message = response_data["result"]["alternatives"][0]["message"]["text"]
        self.messages.append({"role": "assistant", "text": assistant_message})
        return assistant_message

In [ ]:
yandex_gpt = YandexGPT(
    token=os.environ['YANDEX_GPT_IAM_TOKEN'],
    folder_id=os.environ['YANDEX_GPT_FOLDER_ID'],
    system_prompt="""
Ты - профессиональный программист и учитель.
Я пришлю тебе условие задачи, эталонное решение, мое решение с ошибками и описание синтаксических ошибок, если они есть.
Дай очень короткий ответ в одном предложении о синтаксических и логических ошибках в коде моего решения, если они есть.
Объясни ошибку через упоминание смысла задания, не упоминай, как сделано в эталонном решении.
Ни в коем случае не присылай исправленный код моего решения, вообще не присылай код.
Пиши "обратите внимание," или задавай наводящие вопросы, если мне нужно обратить внимание на ошибку в моем коде.
Обращайся ко мне лично и на вы.
"""
)

In [ ]:
# Клеим.

solutions = solutions.merge(tasks[['id', 'description', 'author_solution']], left_on='task_id', right_on='id', how='left', suffixes=('', '_task'))

In [ ]:
# Получение примеров ответов

def get_random_examples(df, n=3):
    examples = df['author_comment'].dropna().unique()
    if len(examples) >= n:
        return random.sample(list(examples), n)
    else:
        return list(examples)

In [ ]:
def create_prompt(row, examples):
    prompt = f"""
Условие задачи:
{row['description']}
Эталонное решение:
{row['author_solution']}
Мое решение:
{row['student_solution']}
Примеры твоего ответа:
"""
    for example in examples:
        prompt += f"{example}\n"
    return prompt

In [ ]:
import time

delay_seconds = 2

submit_df = solutions.copy()
examples = get_random_examples(solutions, n=3)

author_comments = []

for idx, row in tqdm(submit_df.iterrows(), total=submit_df.shape[0], desc="Генерация комментариев"):
    prompt = create_prompt(row, examples)
    try:
        response = yandex_gpt.ask(prompt)
        author_comments.append(response)
    except Exception as e:
        print(f"Ошибка на индексе {idx}: {e}")
        author_comments.append(None)

    # Задержка перед следующим запросом
    time.sleep(delay_seconds)

submit_df['author_comment'] = author_comments

Генерация комментариев: 100%|██████████| 325/325 [22:29<00:00,  4.15s/it]


In [ ]:
submit_df[['id', 'author_comment']].head()

,id,author_comment
0,0,"Обратите внимание на то, что в вашем решении н..."
1,1,Обратите внимание на закрывающую скобку в стро...
2,2,"Обратите внимание на то, что в вашем решении н..."
3,3,"Обратите внимание, что в вашем решении перепут..."
4,4,"Обратите внимание на то, как вычисляется стоим..."
